In [1]:
from data_preprocess import *

In [2]:
import pprint 
pp = pprint.PrettyPrinter(indent=4)

In [3]:
test_cases = [
#     'pglib_opf_case24_ieee_rts.m', 
#     'pglib_opf_case73_ieee_rts.m',
#     'pglib_opf_case162_ieee_dtc.m',
    'pglib_opf_case300_ieee.m',
]

## Example of parsing active constraints: DC-OPF

In [4]:
org_dir = os.getcwd()
os.chdir('./matpower7.0/')

In [5]:
eng = matlab.engine.start_matlab()

In [6]:
test_case_idx = 0
case_name = test_cases[test_case_idx]

In [7]:
data = eng.dc_opf_solver(case_name, 0.03)

In [8]:
Pg = np.squeeze(np.array(data['Pg']))
B_idx = np.squeeze(np.array(data['B_idx']))
GB_map = np.squeeze(np.array(data['GB_map'])).astype(int)
F = np.squeeze(np.array(data['F']))
Pg_lim = np.array(data['Pg_lim'])
F_lim = np.array(data['F_lim'])
w = np.squeeze(np.array(data['w']))

### Generator active constraints: [Pg_min, Pg_max]]

In [9]:
Pg[6]

0.0

In [10]:
Pg_lim

array([[   0.,    0.],
       [   0.,    0.],
       [   0.,    0.],
       [   0.,    0.],
       [   0.,    0.],
       [1448.,    0.],
       [ 185.,    0.],
       [ 360.,    0.],
       [  93.,    0.],
       [ 197.,    0.],
       [2041.,    0.],
       [ 321.,    0.],
       [   0.,    0.],
       [   0.,    0.],
       [ 297.,    0.],
       [ 800.,    0.],
       [  91.,    0.],
       [ 320.,    0.],
       [ 277.,    0.],
       [ 606.,    0.],
       [ 242.,    0.],
       [   0.,    0.],
       [ 330.,    0.],
       [   0.,    0.],
       [ 278.,    0.],
       [ 125.,    0.],
       [ 250.,    0.],
       [2465.,    0.],
       [1624.,    0.],
       [ 515.,    0.],
       [2140.,    0.],
       [ 430.,    0.],
       [ 288.,    0.],
       [ 460.,    0.],
       [ 554.,    0.],
       [ 523.,    0.],
       [ 436.,    0.],
       [ 458.,    0.],
       [ 341.,    0.],
       [ 881.,    0.],
       [ 267.,    0.],
       [ 621.,    0.],
       [1127.,    0.],
       [ 21

In [11]:
Pg_active = get_Pg_active_constraints(Pg, Pg_lim)
Pg_active

array([0., 0., 0., 0., 0., 0., 1., 1., 1., 1., 0., 1., 0., 0., 1., 0., 1.,
       1., 1., 0., 1., 0., 0., 0., 1., 1., 1., 1., 0., 1., 0., 1., 1., 0.,
       1., 1., 1., 1., 1., 1., 1., 1., 0., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 0., 1., 0., 1., 0., 0., 0., 0., 1., 1., 0., 0., 0., 1.,
       1.])

### Bus active constraints, due to precense of co-related generation: [Pb_min, Pb_max]

In [12]:
Pb = get_Pb(Pg, GB_map)
Pb

{8: 0.0,
 10: 0.0,
 20: 0.0,
 63: 0.0,
 76: 0.0,
 84: 853.1783557239853,
 91: 0.0,
 92: 0.0,
 98: 93.0,
 108: 0.0,
 119: 1764.0241713450673,
 124: 321.00000000000006,
 125: 0.0,
 138: 0.0,
 141: 297.0,
 143: 218.2041686157422,
 146: 91.0,
 147: 0.0,
 149: 0.0,
 152: 349.19976696210114,
 153: 242.0,
 156: 0.0,
 170: 51.16634788206274,
 171: 0.0,
 176: 0.0,
 177: 125.0,
 185: 250.0,
 186: 2465.0,
 187: 1623.9999999999998,
 190: 0.0,
 191: 1534.640533670923,
 198: 430.0,
 213: 0.0,
 220: 459.99999999999994,
 221: 554.0,
 222: 0.0,
 227: 436.00000000000006,
 230: 0.0,
 233: 341.00000000000006,
 236: 881.0,
 238: 0.0,
 239: 0.0,
 241: 978.5258310219411,
 242: 0.0,
 243: 342.0,
 7001: 0.0,
 7002: 670.0,
 7003: 1401.0,
 7011: 668.0,
 7012: 626.0,
 7017: 490.00000000000006,
 7023: 0.0,
 7024: 543.0,
 7039: 0.0,
 7044: 122.42503296457494,
 7049: 718.0,
 7055: 150.0,
 7057: 0.0,
 7061: 1214.7840226696821,
 7062: 478.6523493424914,
 7071: 202.99999999999997,
 7130: 1519.9999999999995,
 7139: 0.0,

In [13]:
Pb_lim = get_Pb_lim(Pg_lim, GB_map)
Pb_lim

{8: array([0., 0.]),
 10: array([0., 0.]),
 20: array([0., 0.]),
 63: array([0., 0.]),
 76: array([0., 0.]),
 84: array([1448.,    0.]),
 91: array([185.,   0.]),
 92: array([360.,   0.]),
 98: array([93.,  0.]),
 108: array([197.,   0.]),
 119: array([2041.,    0.]),
 124: array([321.,   0.]),
 125: array([0., 0.]),
 138: array([0., 0.]),
 141: array([297.,   0.]),
 143: array([800.,   0.]),
 146: array([91.,  0.]),
 147: array([320.,   0.]),
 149: array([277.,   0.]),
 152: array([606.,   0.]),
 153: array([242.,   0.]),
 156: array([0., 0.]),
 170: array([330.,   0.]),
 171: array([0., 0.]),
 176: array([278.,   0.]),
 177: array([125.,   0.]),
 185: array([250.,   0.]),
 186: array([2465.,    0.]),
 187: array([1624.,    0.]),
 190: array([515.,   0.]),
 191: array([2140.,    0.]),
 198: array([430.,   0.]),
 213: array([288.,   0.]),
 220: array([460.,   0.]),
 221: array([554.,   0.]),
 222: array([523.,   0.]),
 227: array([436.,   0.]),
 230: array([458.,   0.]),
 233: array([3

In [14]:
Pb_active = get_Pb_active_constraints(Pg, Pg_lim, GB_map)
Pb_active

array([0., 0., 0., 0., 0., 0., 1., 1., 1., 1., 0., 1., 0., 0., 1., 0., 1.,
       1., 1., 0., 1., 0., 0., 0., 1., 1., 1., 1., 0., 1., 0., 1., 1., 0.,
       1., 1., 1., 1., 1., 1., 1., 1., 0., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 0., 1., 0., 1., 0., 0., 0., 0., 1., 1., 0., 0., 0., 1.,
       1.])

### Flow Constraints: [-F_max, F_max] 

In [15]:
F

array([ 5.30576130e+01,  1.01534492e+01,  2.58716501e+01,  1.70325137e+01,
        3.79036465e+01,  2.90815518e+01,  2.71682509e+01, -8.40000000e+01,
        5.55111512e-15,  9.35664884e+00,  8.25750063e+00,  8.25750063e+00,
        6.49128808e+00,  6.49128808e+00,  7.36846965e+00,  9.27514177e-01,
        1.59820112e+00,  1.47353244e+00,  4.57514850e-01,  4.69999327e-01,
        1.14608029e+00,  1.05221613e+00,  7.15835242e+00,  1.94303152e+00,
        1.42200599e+00,  1.95628710e+00,  4.43643416e+00,  4.43643416e+00,
        1.07974208e+00,  8.21031925e-01,  1.67736786e+00,  1.61371565e+00,
        1.68773749e+00,  2.94548214e+00,  1.94662121e+00,  2.77343395e+00,
        4.04993758e+00,  1.37279831e+00,  5.45197582e+01,  1.94884961e+02,
        3.40541789e+02,  1.06490918e+02,  1.57626639e+02,  5.90553462e+01,
        9.95057909e+02, -1.04341530e+02, -5.12521515e+01,  3.71573642e+01,
       -9.94948480e+01,  3.77684198e+02, -1.90035747e+02,  2.37280137e+02,
        4.89287615e+02,  

In [16]:
F_lim

array([[9.900e+03],
       [8.215e+03],
       [6.800e+01],
       [4.600e+01],
       [7.900e+01],
       [7.900e+01],
       [7.800e+01],
       [1.930e+02],
       [3.700e+01],
       [1.170e+02],
       [5.900e+01],
       [5.900e+01],
       [6.700e+01],
       [6.700e+01],
       [3.320e+02],
       [2.100e+01],
       [1.100e+01],
       [9.000e+00],
       [8.000e+00],
       [8.000e+00],
       [1.000e+01],
       [1.500e+01],
       [1.160e+02],
       [7.000e+00],
       [6.000e+00],
       [7.000e+00],
       [3.020e+02],
       [6.110e+02],
       [1.200e+01],
       [6.000e+00],
       [1.800e+01],
       [2.100e+01],
       [9.000e+00],
       [1.800e+01],
       [1.200e+01],
       [1.000e+01],
       [2.900e+01],
       [4.000e+01],
       [4.823e+03],
       [3.240e+03],
       [1.061e+03],
       [9.778e+03],
       [4.230e+02],
       [4.149e+03],
       [1.536e+03],
       [9.910e+02],
       [3.240e+03],
       [4.149e+03],
       [4.820e+02],
       [6.680e+02],


In [17]:
F_active = get_F_active_constraints(F, F_lim)
F_active

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,

### Merge all active constriants

In [18]:
active_aconstraints = merge_active_constraints(Pg_active, Pb_active, F_active)
active_aconstraints

array([0., 0., 0., 0., 0., 0., 1., 1., 1., 1., 0., 1., 0., 0., 1., 0., 1.,
       1., 1., 0., 1., 0., 0., 0., 1., 1., 1., 1., 0., 1., 0., 1., 1., 0.,
       1., 1., 1., 1., 1., 1., 1., 1., 0., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 0., 1., 0., 1., 0., 0., 0., 0., 1., 1., 0., 0., 0., 1.,
       1., 0., 0., 0., 0., 0., 0., 1., 1., 1., 1., 0., 1., 0., 0., 1., 0.,
       1., 1., 1., 0., 1., 0., 0., 0., 1., 1., 1., 1., 0., 1., 0., 1., 1.,
       0., 1., 1., 1., 1., 1., 1., 1., 1., 0., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 0., 1., 0., 1., 0., 0., 0., 0., 1., 1., 0., 0., 0.,
       1., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0.

## (Optional) Example of parsing active constraints: AC-OPF

In [19]:
data = eng.ac_opf_solver(case_name, 0.03)

In [20]:
pp.pprint(data)

{   'B_idx': matlab.double([[1.0],[2.0],[3.0],[4.0],[5.0],[6.0],[7.0],[8.0],[9.0],[10.0],[11.0],[12.0],[13.0],[14.0],[15.0],[16.0],[17.0],[19.0],[20.0],[21.0],[22.0],[23.0],[24.0],[25.0],[26.0],[27.0],[33.0],[34.0],[35.0],[36.0],[37.0],[38.0],[39.0],[40.0],[41.0],[42.0],[43.0],[44.0],[45.0],[46.0],[47.0],[48.0],[49.0],[51.0],[52.0],[53.0],[54.0],[55.0],[57.0],[58.0],[59.0],[60.0],[61.0],[62.0],[63.0],[64.0],[69.0],[70.0],[71.0],[72.0],[73.0],[74.0],[76.0],[77.0],[78.0],[79.0],[80.0],[81.0],[84.0],[85.0],[86.0],[87.0],[88.0],[89.0],[90.0],[91.0],[92.0],[94.0],[97.0],[98.0],[99.0],[100.0],[102.0],[103.0],[104.0],[105.0],[107.0],[108.0],[109.0],[110.0],[112.0],[113.0],[114.0],[115.0],[116.0],[117.0],[118.0],[119.0],[120.0],[121.0],[122.0],[123.0],[124.0],[125.0],[126.0],[127.0],[128.0],[129.0],[130.0],[131.0],[132.0],[133.0],[134.0],[135.0],[136.0],[137.0],[138.0],[139.0],[140.0],[141.0],[142.0],[143.0],[144.0],[145.0],[146.0],[147.0],[148.0],[149.0],[150.0],[151.0],[152.0],[153.0],[154.0

In [21]:
eng.quit()

In [22]:
os.chdir(org_dir)

## Create Datasets for NNs

### example of creating a dataset
- **x**: uncertainty realization as a feature input
- **y**: active constraints index as a label output

In [23]:
dataset = create_dataset(case_name, dataset_size=1)
dataset

> creating dataset with pglib_opf_case300_ieee.m


100%|██████████| 1/1 [00:00<00:00,  1.23it/s]


{'x': array([ 2.15821018e-01, -1.59344805e+00,  2.46894373e-01,  0.00000000e+00,
         9.08338765e+00, -2.48817285e+00,  0.00000000e+00,  1.90197032e-01,
         2.37908160e+00,  2.46096100e+00,  2.23574218e+00,  0.00000000e+00,
        -2.56130534e-01,  4.83731235e+00, -8.07201441e+00,  0.00000000e+00,
        -2.13841046e+01,  0.00000000e+00,  1.17735286e+01,  1.90742971e+00,
        -2.46631305e+00, -2.96774045e-01,  0.00000000e+00, -3.94015557e-01,
         2.53527586e-01,  8.27857052e-01, -1.53443657e+00,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00, -1.60418144e+00, -5.59790238e+00,
         0.00000000e+00, -1.97153267e+00, -5.38126537e-02,  0.00000000e+00,
         2.54800109e+00,  6.66002252e+00,  0.00000000e+00,  0.00000000e+00,
        -2.43275990e+00, -3.13717871e-01,  4.53755242e-01,  1.12160104e-01,
        -4.99675917e-01,  3.26294130e+00, -1.44281146e-01,  2.16158000e-01,
         1.95431831e+00,  3.57792089e-02,  5.76143122e+00,  0.00000000e+00,
       

### Build datasets

In [24]:
# dataset_size = 2
dataset_size = 50000

In [25]:
build_datasets(test_cases, dataset_size)

> creating dataset with pglib_opf_case300_ieee.m


100%|██████████| 50000/50000 [5:36:08<00:00,  2.48it/s]  


### Check datasets

In [26]:
infile = open('./datasets/pglib_opf_case24_ieee_rts.pickle','rb')
dataset = pickle.load(infile)
infile.close()

In [27]:
dataset

{'x': array([[ 1.58401582,  3.01095666,  3.92517972, ...,  0.        ,
          0.        ,  0.        ],
        [ 3.54218077,  3.2279853 , -4.66372524, ...,  0.        ,
          0.        ,  0.        ],
        [-0.62343597,  2.58585634, -4.13018588, ...,  0.        ,
          0.        ,  0.        ],
        ...,
        [-1.73786221, -1.83840692,  6.28492785, ...,  0.        ,
          0.        ,  0.        ],
        [-2.94739091, -1.37913875, -7.33716418, ...,  0.        ,
          0.        ,  0.        ],
        [-1.56552295,  3.44187377,  8.9611732 , ...,  0.        ,
          0.        ,  0.        ]]),
 'y': array([[1., 1., 1., ..., 0., 0., 0.],
        [1., 1., 1., ..., 0., 0., 0.],
        [1., 1., 1., ..., 0., 0., 0.],
        ...,
        [1., 1., 1., ..., 0., 0., 0.],
        [1., 1., 1., ..., 0., 0., 0.],
        [1., 1., 1., ..., 0., 0., 0.]])}